In [9]:
import pandas as pd
import os
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)



In [10]:
seg_folder = 'outputs/metrics_ondri_fs/'
metric_df = []
seg_files = os.listdir(seg_folder)
for file in seg_files:
    metric_df.append(pd.read_csv(seg_folder + file))

metric_df = pd.concat(metric_df)
metric_df['SUBJECT'] = [x.split('.')[:-1][0] for x in seg_files]
# if 'TBV-voxel' in metric_df.columns:
#     metric_df = metric_df.drop(columns=['TBV-voxel'])

metric_df = metric_df.dropna(axis=1, how='all')


In [11]:
nvox_cols = metric_df.columns[metric_df.columns.str.contains('nvox')]
metric_df = metric_df.drop(columns=nvox_cols)
#metric_df[nvox_cols] = np.asarray(metric_df[nvox_cols])/np.expand_dims(metric_df['TBV-voxel'], axis=-1)

In [12]:
delete_cols = metric_df.columns[metric_df.columns.str.contains('MII')  | metric_df.columns.str.contains('mean')]
delete_cols = delete_cols + ['TBV-voxel']
metric_df = metric_df.drop(delete_cols, axis=1)

In [13]:
clinical_df = pd.read_csv('data/summary/ONDRI_summary.csv')
clinical_df = clinical_df[~pd.isna(clinical_df['NII_FILENAME_T1'])]

In [14]:
fs_t1_vols = pd.read_csv('outputs/fs_t1_volumes.csv')
nvox_cols = fs_t1_vols.columns[~fs_t1_vols.columns.str.contains('SUBJECT')]
fs_t1_vols[nvox_cols] = fs_t1_vols[nvox_cols].to_numpy()/np.expand_dims(fs_t1_vols['total_intracranial_volume'].to_numpy(), axis=-1)
fs_t1_vols = fs_t1_vols.rename(columns={x: x + '_nvox' for x in nvox_cols})
merged = pd.merge(clinical_df[['SUBJECT', 'COHORT']], metric_df, on='SUBJECT')

merged = pd.merge(merged, fs_t1_vols, on='SUBJECT', how='inner')

In [15]:
if not os.path.exists('outputs/analysis'):
    os.makedirs('outputs/analysis')

In [16]:
# train models
from sklearn.model_selection import KFold
from imblearn.ensemble import BalancedRandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  recall_score, roc_auc_score, classification_report
from sklearn.metrics import confusion_matrix as confusion_matrix_fn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from scipy.stats import ttest_ind
import shap





for version in 'vol', 'texture', 'both':
    comparisons = [('ADMCI', 'FTD'), ('FTD', 'ALS'), ('ADMCI', 'PD')]
    classification_accuracy_df = []

    for comparison in comparisons:
        print(comparison)
        labelmap = {comparison[0]: 0, comparison[1]: 1}

        merged_select = merged[merged['COHORT'].isin(list(labelmap.keys()))]
        features = merged_select[list(filter(lambda x: x not in [ 'SUBJECT', 'COHORT'], merged_select.columns))]
        features = features.dropna(axis=1, how='any')
  
        if version == 'vol':
            vol_cols = list(filter(lambda x: 'nvox' in x, features.columns))

            features = features[vol_cols]

        elif version == 'texture':
            mii_cols = list(filter(lambda x: 'MAD' in x, features.columns)) 
            features = features[mii_cols]
        #features = features.drop(columns=['TBV-voxel'])

        labels = merged['COHORT']
        labels = merged_select['COHORT'].map(labelmap)

        kf = KFold(n_splits=10, shuffle=True, random_state=49)

        val_predictions_lr = np.zeros((labels.shape[0], 2))
        val_predictions_rf = np.zeros((labels.shape[0], 2))
        models_lr = []
        shap_lr = []
        models_rf =[]
        shap_rf = []

        for train_idx, val_idx in kf.split(features):
            train_features = features.iloc[train_idx]
            train_labels = labels.iloc[train_idx]
            val_features = features.iloc[val_idx]
            val_labels = labels.iloc[val_idx]

            # ---------------------------------------------------------------------------------------------------------------------------------------------
            # Random Forest training
            model = BalancedRandomForestClassifier(random_state=0)
            model.fit(train_features, train_labels)

            explainer = shap.TreeExplainer(model, data=val_features)
            shap_values = explainer.shap_values(val_features, check_additivity=False)

            shap_rf.append(shap_values)
            models_rf.append(model)
            val_predictions_rf[val_idx] = model.predict_proba(val_features)


        coef_df = pd.DataFrame()
        coef_df['feature'] = features.columns
        mean_imps = np.mean(np.array([model.feature_importances_ for model in models_rf]), axis=0)
        mean_shaps = np.mean(np.abs(np.concatenate([shap_values[1] for shap_values in shap_rf])), axis=0)

        coef_df['shap'] = mean_shaps
        coef_df['imps'] = mean_imps
        coef_df['mean_percent'] = [(np.mean(features[x][labels==1])/np.mean(features[x][labels==0])) for x in coef_df['feature']]
        coef_df['p'] = [ttest_ind(features[x][labels==0], features[x][labels==1]).pvalue for x in coef_df['feature']]
        coef_df = coef_df.sort_values(by='shap', ascending=False)
        coef_df.to_csv('outputs/analysis/fs_%s_vs_%s_%s.csv' % (comparison[0], comparison[1], version), index=False)


        confusion_matrix = confusion_matrix_fn(labels, np.argmax(val_predictions_rf, axis=-1))

        recall_0 = confusion_matrix[0, 0]/ (confusion_matrix[0, 0] + confusion_matrix[0, 1]) # (yes this agrees with sklearn confusion_matrix)
        recall_1 = confusion_matrix[1, 1]/ (confusion_matrix[1, 0] + confusion_matrix[1, 1])

        classification_accuracy_df.append(pd.Series({'comparison': '%s vs %s' % (comparison[0], comparison[1]), 
                                                    'auc': roc_auc_score(labels, val_predictions_rf[:, 1]),
                                                    'Recall 0': recall_0,
                                                    'Recall 1': recall_1,
                                                    })) 
                                                    
                                                                    
    classification_accuracy_df = pd.DataFrame(classification_accuracy_df)

    classification_accuracy_df['Recall 0'] = np.round(classification_accuracy_df['Recall 0'], 2)
    classification_accuracy_df['Recall 1'] = np.round(classification_accuracy_df['Recall 1'], 2)
    classification_accuracy_df['auc'] = np.round(classification_accuracy_df['auc'], 2)
    classification_accuracy_df.to_csv('outputs/analysis/classification_accuracy_fs_%s.csv' % version, index=False)

('ADMCI', 'FTD')


Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.


('FTD', 'ALS')


Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.


('ADMCI', 'PD')


...

Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.


('FTD', 'ALS')


Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.


('ADMCI', 'PD')


Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
